In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from tensorflow.keras.datasets import cifar10

# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [ ]:
# 이미지 값 변환
train_images_2 = np.array(train_images/255.0, dtype=np.float32)
test_images_2 = np.array(test_images/255.0, dtype=np.float32)

# 라벨 값 변환
train_labels_2 = np.array(train_labels, dtype=np.float32)
test_labels_2 = np.array(test_labels, dtype=np.float32)

In [ ]:
# 동일하게 원핫이나 이런거 해도 된다 -> 입출력 데이터의 차원을 알기만 하면 된다
train_labels_2 = train_labels.squeeze()
test_labels_2 = test_labels.squeeze()

# from tensorflow.keras.utils import to_categorical
# train_labels_2 = to_categorical(train_labels_2)
# test_labels_2 = to_categorical(test_labels_2)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_images_2, train_labels_2, test_size=0.2, random_state=1)
train_x.shape, val_x.shape, train_y.shape, val_y.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (40000,), (10000,))

사전훈련 모델 불러오기

In [ ]:
import tensorflow.keras.applications

In [ ]:
dir(tensorflow.keras.applications)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications import ResNet50V2

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# include_top=False로 기존 imagenet용 classifier 층들을 다 제거
# weight는 전이학습을 위해 imagenet 학습된 weight를 초기 weight로 사용

# 이쪽 분야는 사전 훈련 모델을 쓸때 base라고 한다
# 파라미터 1. 입력모양, 2. 분류기 가져올 건지


# 3. 이미 학습된 가중치도 가져오니 더 이상 건들이지 말라는 의미(이미 학습된 애를 가져와서 그대로 사용할 거다란 뜻) = vgg의 파라미터는 오차역전파로 수정하지 않겠다 -> 바꾸려면 None로 하고 에폭을 엄청 늘려야함
# + w는 이미지넷이라는 w를 가져온다

base_model = VGG16(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
# base_model = DenseNet121(input_shape=(32, 32, 3), include_top=False, weights='imagenet')




# 이모델은 flatten하고 dense하고 있는게 생략 되어있다 이걸 처리해 줘야한다 = 마지막 레이어를 처리해야함 = 미완성됨 모델을 내가 만드는 거임
bm_output = base_model.output # 모델의 끝부분이라는 의미

In [ ]:
# 컨브 레이어만 w가 100만개가 넘는다
base_model.summary()

In [ ]:
# 글로벌 맥스풀링은 각 공간 차원이 1이 될 때까지 공간 차원에 평균 풀링을 적용하고 다른 차원은 변경하지 않고 그대로 둚.
# Flatten과 비슷한 기능을 함


#flatten이 단순하게 한줄로 세운다면 얘는 평균을 계산하면서 압축을 하면서 줄을 세움 = 계산식이 들어간 Flatten이다 = 전이 학습이 너무 깊으니깐 단순히 줄을 세우는게 아니라 압축을 하면서 줄을 세우자 이거임
#flatten을 써도 되지만 너무 깊을 때는 이걸 쓰는게 훨씬 성능이 좋음
output_layer = GlobalAveragePooling2D()(bm_output)
output_layer = Dense(50, activation='relu')(output_layer)

# 역시 마지막은 클래스 개수에 맞추는 거고 + 이걸 봤을때 flatten하고 dense하는 걸 처리한다는 느낌이 확듬
output_layer = Dense(10, activation='softmax')(output_layer)

model = Model(inputs=base_model.input, outputs=output_layer)# 모델의 시작부분이라는 의미

In [ ]:
# 사전 훈련 모델은 모델이 너무 깊어서 과적합을 줄이기 위해 학습률을 조정하여 과적합 방지를 도와줄 수 있다
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# 아까는 학습이 빨리 수렴됐는데 이번에는 엄청 오래걸린다

batch_size = 64
epochs = 10

history = model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_x, val_y))

Epoch 1/20
625/625 [==============================] - 25s 38ms/step - loss: 1.8311 - accuracy: 0.2724 - val_loss: 1.5426 - val_accuracy: 0.4042
Epoch 2/20
625/625 [==============================] - 23s 37ms/step - loss: 1.2896 - accuracy: 0.5121 - val_loss: 1.1533 - val_accuracy: 0.5894
Epoch 3/20
625/625 [==============================] - 23s 37ms/step - loss: 1.0071 - accuracy: 0.6404 - val_loss: 1.0296 - val_accuracy: 0.6488
Epoch 4/20
625/625 [==============================] - 24s 38ms/step - loss: 0.8373 - accuracy: 0.7093 - val_loss: 0.8474 - val_accuracy: 0.7040
Epoch 5/20
625/625 [==============================] - 24s 38ms/step - loss: 0.7147 - accuracy: 0.7549 - val_loss: 0.8229 - val_accuracy: 0.7259
Epoch 6/20
625/625 [==============================] - 24s 38ms/step - loss: 0.6125 - accuracy: 0.7946 - val_loss: 0.7701 - val_accuracy: 0.7521
Epoch 7/20
625/625 [==============================] - 24s 38ms/step - loss: 0.5521 - accuracy: 0.8174 - val_loss: 0.8162 - val_accuracy:

In [ ]:
# 테스트 데이터로 성능 평가
model.evaluate(test_images_2, test_labels_2)

313/313 [==============================] - 3s 9ms/step - loss: 0.9692 - accuracy: 0.7910


[0.9692130088806152, 0.7910000085830688]

In [ ]:
# 결과 한 에폭당 25초
# 정확도 : 점점 수렴하다가 79에서 멈춤 -> cnn cifar10이 74가 나왔는데 이모델을 사용하니 내가 굳이 딥러닝 층을 구현하지 않고 가볍게 썼는데 오히려 성능이 더 좋아짐!!
# 앞으로 코드를 다 구현하는 일은 없을 거다 나에게 맞는 모델을 찾아서 불러와서 돌리는 경우가 훨씬 많고 오히려 성능이 더 좋을 것이다 but 그 모델을 내 맘대로 바꾸는데 모델 구성법을 알아야한다